# Import modules

In [1]:
import requests
import pandas as pd
import numpy as np
#from requests import get
from bs4 import BeautifulSoup
from selenium import webdriver
from time import sleep
from random import randint

# Initialise storage

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36'}
Name = []
Address = []
InfoLink = []
Telephone=[]

# First loop to get name, address, and details url

In [3]:
pages = np.arange(0, 100, 10)
for page in pages: 

  page = requests.get("https://reports.ofsted.gov.uk/search?q=&location=&radius=&level_2_types%5B0%5D=6&level_2_types%5B1%5D=7&status%5B0%5D=1&level_1_types=2&start=" + str(page) + "&rows=10", headers=headers)

  soup = BeautifulSoup(page.text, 'html.parser')
  nursery_li= soup.find_all('li', class_='search-result')
    
  
  sleep(randint(2,10))

  for container in nursery_li:
        
        name = container.find('h3', class_='search-result__title heading--main').text
        Name.append(name)

        address = container.find('address', class_='search-result__address').text
        Address.append(address)
        
        # get urls
        #urls = [item.get("href") for item in soup.find_all("a")]
        
        nextURL = container.find('a').attrs['href']
        nextURL = "https://reports.ofsted.gov.uk" + str(container.find('a').attrs['href'])
        InfoLink.append(nextURL)
    


# Second loop to get phone numbers which are on the page of the embedded url

In [4]:
for i in range(0,100):
    url = InfoLink[i]
    driver2 = webdriver.Chrome()
    driver2.get(url)  
    sleep(randint(10,20))
    soup = BeautifulSoup(driver2.page_source, 'html.parser')

    
    my_table2 = soup.find(class_=['info-block__items list-unstyled'])
    
    
    tel=my_table2.find_all(class_='col half')[-1]
    


    for t in tel:
        Telephone.append(t.string.strip())


In [5]:
Telephone

['01264535053',
 '01454 862233',
 '07791051001',
 '01614421113',
 '01454 772117',
 '07799541168',
 '07712 212070',
 '07814418478',
 '07738851790',
 '07732127731',
 '07732129842',
 '07703448566',
 '07918187430',
 '07732133378',
 '07716756152',
 '0208 549 9995',
 '0207 974 7038',
 '02036673327',
 '0207635 6408',
 '01536 264566',
 '01704893279',
 '01634382330',
 '01634 578697',
 '02077408070',
 '0203 841 9000',
 '0207 7408070',
 '02036673277',
 '02085098372',
 '0151 922 6594',
 '01519441345',
 '02085933063',
 '07968238922',
 '07946246022',
 '02036673277',
 '02083854390',
 '07905055967',
 '07939244564',
 '01772780809',
 '01254681899',
 '03330 432222',
 '01582455105',
 '07849 097247',
 '07842088646',
 '02086850673',
 '0191 447 7877',
 '0191 4873297',
 '0191 4111300',
 '07772585191',
 '01422 835658',
 '020 75869871',
 '0117 9863290',
 '07944 220 280',
 '01932 569326',
 '02084442950',
 '02084440684',
 '01252 918888',
 '01252 918888',
 '01252 918888',
 '01604587302',
 '020 8677 7088',
 '020826

In [6]:
nurseries = pd.DataFrame({
'Nurseries': Name,
'Addresses': Address,
#'Links': InfoLink,
'Contact': Telephone,    
})

# Remove spaces in phone numbers

In [7]:
nurseries['Contact'] = nurseries['Contact'].apply(lambda x: x.replace(' ', '') if ' ' in str(x) else x)

In [8]:
nurseries['Contact']

0     01264535053
1     01454862233
2     07791051001
3     01614421113
4     01454772117
         ...     
95    02077319670
96    01132757698
97    02084519605
98    02088472547
99    01214294112
Name: Contact, Length: 100, dtype: object

# Print

In [9]:
# to see your dataframe
print(nurseries)

# to see the datatypes of your columns
print(nurseries.dtypes)

# to see where you're missing data and how much data is missing 
print(nurseries.isnull().sum())

# to move all your scraped data to a CSV file
nurseries.to_csv('nurseries.csv')

                                       Nurseries  \
0                   'Andover Kumon Study Centre'   
1                  'The Zone' Out Of School Club   
2          (K.A.O.S.) Upminster Methodist Church   
3                              0-2-5 Nursery LTD   
4                      100 Acre Wood Day Nursery   
..                                           ...   
95  57 Filmer Road,Private Nursery and Preschool   
96                         79 2 81 Childcare Ltd   
97                        8 till 6 Eagle Nursery   
98                              9 Months Nursery   
99            A B C Day Nursery & Pre School Ltd   

                                            Addresses      Contact  
0   Unit 23-24 Walworth Enterprise Centre, West Wa...  01264535053  
1   St Barnabas C Of E Primary School, Poplar Clos...  01454862233  
2   Wesleyan Methodist Church, Hall Lane, UPMINSTE...  07791051001  
3    22 Thornfield Road, STOCKPORT, Cheshire, SK4 3JT  01614421113  
4   149 Church Road, Frampton 